In [ ]:
# from google.colab import drive

# drive.mount('/content/drive')


In [ ]:
# pip install tflearn

In [27]:
import json
import numpy as np
import tensorflow as tf
import nltk
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Activation, Dropout
# from tensorflow.keras.optimizers import SGD
json_path = 'intents.json'

In [28]:
with open(json_path) as intention:
    intents = json.load(intention)
# print(intents)

In [29]:
words = []
tag = []
docs = []
ignore=['!','?','.....',',','.']

In [ ]:
nltk.download('all')

In [30]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        token = nltk.word_tokenize(pattern)
        words.extend(token)
        docs.append((token,intent['tag']))
        if intent['tag'] not in tag:
            tag.append(intent['tag'])



In [ ]:
print(words)
print(tag)
print(docs)

In [31]:
lemmatizer = nltk.WordNetLemmatizer()
words = [lemmatizer.lemmatize(token.lower()) for token in words if token not in ignore]
words = sorted(list(set(words)))

In [ ]:
wordss = np.array(words)
print(wordss.shape)

In [45]:
training_list = []
output_nan = [0] * len(tag)
# bow_shape = np.array(words).shape

for doc in docs:
    bow = []
    pattern = doc[0]
    pattern = [lemmatizer.lemmatize(sentence.lower()) for sentence in pattern]
    for word in words:
        bow.append(1) if word in pattern else bow.append(0)

    output = list(output_nan)
    output[tag.index(doc[1])] = 1
    bow_array = np.array(bow)
    # bow_array = np.pad(bow_array, (0, bow_shape[0] - len(bow_array)))
    output_array = np.array(output)

    training_list.append((bow_array, output_array))

# train_x = np.stack([item[0] for item in training_list])
# train_y = np.stack([item[1] for item in training_list])
training = np.array(training_list)
train_x = list(training[:,0])
train_y = list(training[:,1])

train_x = np.stack(train_x)
train_y = np.stack(train_y)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (31, 2) + inhomogeneous part.

In [33]:
print(train_x.shape)

(31, 59)


In [34]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, input_shape=(len(train_x[0]), )),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(len(train_y[0]), activation='softmax')
])

model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer='adam', metrics=['acc'])
model.fit(train_x, train_y, epochs=1000, batch_size=8)

Epoch 1/1000
4/4 [==============================] - 0s 2ms/step - loss: 1.9856 - acc: 0.2581
Epoch 2/1000
4/4 [==============================] - 0s 3ms/step - loss: 1.9704 - acc: 0.2581
Epoch 3/1000
4/4 [==============================] - 0s 5ms/step - loss: 1.9570 - acc: 0.2581
Epoch 4/1000
4/4 [==============================] - 0s 3ms/step - loss: 1.9441 - acc: 0.2581
Epoch 5/1000
4/4 [==============================] - 0s 2ms/step - loss: 1.9315 - acc: 0.2903
Epoch 6/1000
4/4 [==============================] - 0s 2ms/step - loss: 1.9186 - acc: 0.3226
Epoch 7/1000
4/4 [==============================] - 0s 3ms/step - loss: 1.9070 - acc: 0.3226
Epoch 8/1000
4/4 [==============================] - 0s 4ms/step - loss: 1.8942 - acc: 0.3226
Epoch 9/1000
4/4 [==============================] - 0s 2ms/step - loss: 1.8822 - acc: 0.3226
Epoch 10/1000
4/4 [==============================] - 0s 1ms/step - loss: 1.8705 - acc: 0.3548
Epoch 11/1000
4/4 [==============================] - 0s 2ms/step - lo

In [ ]:
model.save('chatbot_model_4.h5')

In [ ]:
load_model = tf.keras.models.load_model('chatbot_model_4.h5')


In [ ]:
load_model.summary()

In [35]:
with open(json_path) as f:
  intents = json.load(f)

In [ ]:
print(intents)

In [36]:
new_word = []
classes = []
documents = []
ignore_words = ['?', '!','.....',',','.']

In [37]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        token = nltk.word_tokenize(pattern)
        new_word.extend(token)
        documents.append((token,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])



In [38]:
def clean_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words if word not in ignore]
    sentence_words = sorted(list(set(sentence_words)))
    return sentence_words

In [39]:
def bag_of_words(sentence, words, show_details=False):
        sentence_words = clean_sentence(sentence)
        bag = [0] * len(words)
        for s in sentence_words:
            for i, w in enumerate(words):
                if w == s:
                    bag[i] = 1

        arrBag = np.array(bag)
        return np.stack(arrBag)

In [43]:
import random
def generate_response(sentence):
        input_data = bag_of_words(sentence, words)
        input_data = input_data.reshape(1, input_data.shape[0])
        results = model.predict(input_data)[0]
        results_index = np.argmax(results)
        tag = classes[results_index]

        if results[results_index] > 0.5:
            for intent in intents['intents']:
                if intent['tag'] == tag:
                    response = random.choice(intent['responses'])
                    # links = intent.get('links', [])
                    return response
        else:
            return "I'm sorry, I didn't understand that.", []

In [40]:
print(bag_of_words('P', words).shape)

(59,)


In [ ]:
print(np.array(words).shape)

In [44]:
print("Chatbot: Hello! How can I assist you?")
while True:
        user_input = input("User: ")
        if user_input.lower() == 'quit':
            print("Chatbot: Goodbye!")
            break
        response= generate_response(user_input)
        print("Chatbot:", response)

Chatbot: Hello! How can I assist you?
1/1 [==============================] - 0s 18ms/step
Chatbot: Rasa kopi bisa dipengaruhi oleh teknik brewing nya, atau level roast nya, dan juga oleh jenis beans nya. Berikut adalah tautan-tautan yang bisa memberi kamu informasi lebih jauh
1/1 [==============================] - 0s 18ms/step
Chatbot: Hmmm.... saya menemukan tautan yang bisa membantu, yuk dicek!
1/1 [==============================] - 0s 17ms/step
Chatbot: Kafein di dalam kopi bisa beragam, dipengaruhi oleh jenis beans yang digunakan ataupun metode pengolahannya. Untuk informasi lebih lengkap, kamu bisa cek tautan-tautan berikut
1/1 [==============================] - 0s 17ms/step
Chatbot: Rasa kopi bisa dipengaruhi oleh teknik brewing nya, atau level roast nya, dan juga oleh jenis beans nya. Berikut adalah tautan-tautan yang bisa memberi kamu informasi lebih jauh
1/1 [==============================] - 0s 15ms/step
Chatbot: Banyak sekali brand kopi yang terkenal di Indonesia. Namun jika